# Spam Filter with Naive Bayes

## Introduction

Spam messages are disturbing and misguiding in our daily lives. It would be helpful if there is a system that could learn from a dataset where messages are classified and use that knowledge to classify new messages as spam or non-spam. In this project we aim to build such a system using the Naive Bayes Algorithm. 

Specifically, we are going to use a multinomial Naive Bayes algorithm with a dataset of 5,572 SMS messages that are already classified. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo and it could be downloaded at [ The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)

In [1]:
import pandas as pd

messages = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
print(messages.shape)

(5572, 2)


In [2]:
messages.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


We see that the dataset has two columns and 5527 rows as expected. The message is either classified as ham (not spam) or spam.

In [3]:
messages['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Of all messages, 86.6% are not spam messages while 13.4% are spam.

## Testing

When developing a software like a spam filter it is important to build a test before building. Building a test prior allows us to create a unbiased environment which indicates the efficiency of our software. We are going to use 80% of our data (4,458 messages) as our training set and the rest (1,114 messages) as our test set.

In [4]:
# randomize the entire dataset
data_randomized = messages.sample(frac = 1, random_state = 1)

# calculate index for split
training_test_index = round(len(data_randomized)*0.8)

# training/test split
training_set = data_randomized[:training_test_index].reset_index(drop = True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)
training_set['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [5]:
test_set['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

The percentage of spam to ham is similar to our original data, therefore this sample is repesentitive. 

## Data Cleaning

To apply the Naive Bayes algorithm, we have to take into account every single word. We have to reorganize our data into a more usable format and make everything lowercase and punctuation-less.

In [6]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


the **vocabulary**  is the set of unique words. To work with the Naive Bayes algorithm, we would have to extract the vocabulary of our training set.

In [7]:
training_set['SMS'] = training_set['SMS'].str.split(' ')

vocabulary = []
for column in training_set['SMS']:
    for word in column:
        vocabulary.append(word)
        
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)
print(len(vocabulary))

7784


In [8]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [9]:
word_count = pd.DataFrame(word_counts_per_sms)
training_set_clean = pd.concat([training_set, word_count], axis=1)
training_set_clean.head()

,Label,SMS,,85023,tkls,poor,hillsborough,facilities,hilarious,08715205273,...,machi,geoenvironmental,jokes,renewing,online,details,schedule,cheat,acid,luks
0,ham,"[yep, , by, the, pretty, sculpture]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, , princess, , are, you, going, to, make,...",3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[havent, ]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, , , there, s...",7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants

Now that we have cleaned our data, we could finally start developing our model. There are several probability that we could calculate before writing our full probability calculation.

In [10]:
total_messages = len(training_set_clean.index)

# Probability of Ham Message
ham_sms = training_set_clean[training_set_clean['Label'] == 'ham']
ham_sms_length = len(ham_sms.index)
p_ham = ham_sms_length / total_messages

# Probability of Spam Message
spam_sms = training_set_clean[training_set_clean['Label'] == 'spam']
spam_sms_length = len(spam_sms.index)
p_spam = spam_sms_length / total_messages

print(p_ham)
print(p_spam)

0.8654104979811574
0.13458950201884254


In [11]:
# Number of total words in spam, ham, and vocabulary
n_vocabulary = len(vocabulary)

n_words_per_spam_message = spam_sms['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

n_words_per_ham_message = spam_sms['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# Laplace Smoothing
alpha = 1

## Calculate Parameter

The Naive Bayes algorithm contains a couple parameters: P(word|spam) and P(word|ham). If we don't change our training set, those values stay constant. To make our algorithm fast, we could calcualte those values seperate from the classification. 

In [12]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_sms[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_sms[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Classifying A Message

In [13]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')


Now that we have our classifier, we can test a couple messages to make sure our algorithm is working properly on individual bases.

In [14]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 4.8441099043219405e-26
P(Ham|message): 8.110202146876641e-24
Label: Ham


In [15]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 1.099416001503406e-25
P(Ham|message): 2.419939851641262e-18
Label: Ham


The first message is clearly a spam message, however, our classifier labeled the message as being a ham. Before we conclude that our classfier is inefficient, let's check it with our test set and determine the acuracy under more iterations.

## Calculating Filter Accuracy

In [19]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [20]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [26]:
# Calculate Accuracy

correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
accuracy = correct/total

print('Correct:', correct)
print('Incorrect:', total-correct)
print('Accuracy:', correct/total)

Correct: 1058
Incorrect: 56
Accuracy: 0.9497307001795332


Despite it incorrectly classifying 